## Analysis of datasets + stats for compressing with FPDE
This notebook contains methods for passing a .shp-file (or directory) and obtaining stats for the FPDE properties of the compression. Stats include the number of chunks, number of deltas within each chunk, average number of vertices, and a distribution of the overhead within the format.

## Evaluating one shape-file
The methods take one .shp-file and return the stats. Multiple calls can be merged further done if a set of .shp-files are to be analysed.

In [10]:
import jsonlines
import glob
import osmnx as ox
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import folium
import tqdm
import shapely
import random
import os
from shapely.wkt import loads
from algos.fpd_extended_lib.cfg import *
import numpy as np
import re
import seaborn as sns
from algos.alg_fpd_extended import FpdExtended

In [11]:
#df = gpd.read_file('data/ne_10m_admin_1_states_provinces.shp')
#df = gpd.read_file('data/sweden-latest-free/gis_osm_buildings_a_free_1.shp')
#df = gpd.read_file('data/sweden-latest-free/gis_osm_natural_a_free_1.shp')
MAX_ITER = -1

def load_shp(file_name, DO_VERTEX_CNT):
    df = gpd.read_file(file_name)
    print("Count of entries:", len(df))
    df = df[df.type != "Point"]
    print("---", file_name, "---")
    print("Count of entries (no points):", len(df))
    shapes = df.geometry

    vertex_cnt = []
    if DO_VERTEX_CNT:
        for s in tqdm.tqdm(shapes[0:MAX_ITER]):
            vertex_cnt.append(shapely.get_num_coordinates(s))

        vertex_cnt = np.array(vertex_cnt)
        if len(vertex_cnt) != 0:
            res = f"Total: {len(vertex_cnt)}, avg: {round(np.average(vertex_cnt), 2)}, std: {round(np.std(vertex_cnt), 2)}, median: {np.median(vertex_cnt)}, min: {np.min(vertex_cnt)}, max: {np.max(vertex_cnt)}, #>=25: {(vertex_cnt >= 25).sum()} ({round(100 * (vertex_cnt >= 25).sum() / len(vertex_cnt), 2)}), #>=100: {(vertex_cnt >= 100).sum()} ({round(100 * (vertex_cnt >= 100).sum() / len(vertex_cnt), 2)}), #>=200: {(vertex_cnt >= 200).sum()} ({round(100 * (vertex_cnt >= 200).sum() / len(vertex_cnt), 2)}), #>=500: {(vertex_cnt >= 500).sum()} ({round(100 * (vertex_cnt >= 500).sum() / len(vertex_cnt), 2)}), #>=1000: {(vertex_cnt >= 1000).sum()} ({round(100 * (vertex_cnt >= 1000).sum() / len(vertex_cnt), 2)})"
            print(res)
            with open('vertex_cnt_for_datasets.txt', 'a') as f:
                f.write(f"{file_name}\n{res}\n\n")
        else:
            print("No geometries (except Point(s)) in dataset.")
    return shapes

In [12]:
def fpde_analyze(shapes):
    stats = pd.DataFrame(columns=["No Comp Size", "Comp Size", "Comp Factor", "Vertex Cnt", "Chunk Cnt", "Avg Vertices in Chk", "Min Vertices in Chk", "Max Vertices in Chk"])
    stats_distribution = pd.DataFrame()
    stats_max_values = pd.DataFrame()
    alg = FpdExtended()

    for idx, s in enumerate(tqdm.tqdm(shapes[0:MAX_ITER])):
        bin = alg.compress(s)[1]
        wkb_len = len(shapely.to_wkb(s))
        bin_len = len(bin)
        coords_len = shapely.get_num_coordinates(s)

        chks, _, overhead_stats = alg.get_chunks(bin, include_ring_start=False, verbose=True)
        chk_cnt = len(chks)
        chk_lens = list(map(lambda x: len(x), chks))

        stats.loc[len(stats)] = [wkb_len, bin_len, wkb_len / bin_len, coords_len, chk_cnt, np.average(chk_lens), np.min(chk_lens), np.max(chk_lens)]

        # Analysis of space
        max_values, distrb = overhead_stats
        theoretical_size = sum(distrb.values())
        distrb = pd.DataFrame(distrb, index=[idx])
        stats_distribution = pd.concat([stats_distribution, distrb])
        stats_max_values = pd.concat([stats_max_values, pd.DataFrame(max_values, index=[idx])])

        DISPLAY_PER_SHAPE_STATS = False
        if DISPLAY_PER_SHAPE_STATS:
            display(distrb)
            print("Size (calculated, rounded to byte, real):", theoretical_size, f"({((theoretical_size + 7) & (-8))})", bin_len * 8)
            print("Max Values:", dict(max_values))

    print("---- AVERAGE COMPRESSED SHAPE ----")
    pd.set_option('display.precision', 2)
    display(stats.mean())
    print("Global: Min Vertices in Chk / Max Vertices in Chk", np.min(stats['Min Vertices in Chk']), np.max(stats['Max Vertices in Chk']))

    print("---- DATA DISTRIBUTION ----")
    final_distribution = stats_distribution.mean()
    display(final_distribution)
    display(final_distribution.apply(lambda x: 100 * x / sum(final_distribution)))

    print("---- TO SET MANUAL PARAMS ----")
    final_max_values = stats_max_values.max()
    min_bits = final_max_values.apply(lambda x: required_bits(x))
    min_bits = min_bits.add_prefix("Bits ")
    display(pd.concat([final_max_values, min_bits]))

In [18]:
DATASET = 'data/sweden-latest-free/gis_osm_railways_free_1.shp'
#DATASET = 'data/sweden-latest-free'
#DATASET = 'data/sweden-latest-free/gis_osm_natural_free_1.shp'
DATASET = "data/new-york-latest-free"
ONLY_VERTEX_CNT = True # Can be used if only the amount of vertices in the dataset is required

if DATASET.endswith(".shp"):
    files = [DATASET]
else:
    files = glob.glob(DATASET + '/*.shp')

for f in tqdm.tqdm(files):
    shapes = load_shp(f, ONLY_VERTEX_CNT)
    if not ONLY_VERTEX_CNT:
        fpde_analyze(shapes)

  0%|          | 0/18 [00:00<?, ?it/s]

Count of entries: 131910
--- data/new-york-latest-free/gis_osm_landuse_a_free_1.shp ---
Count of entries (no points): 131910


  6%|▌         | 1/18 [00:28<08:06, 28.62s/it]

Total: 131909, avg: 31.9, std: 125.63, median: 16.0, min: 4, max: 16897, #>=25: 39826 (30.19), #>=100: 5861 (4.44), #>=200: 2073 (1.57), #>=500: 501 (0.38), #>=1000: 156 (0.12)
Count of entries: 57458
--- data/new-york-latest-free/gis_osm_natural_free_1.shp ---
Count of entries (no points): 0


0it [00:00, ?it/s]
 11%|█         | 2/18 [00:35<04:10, 15.68s/it]

...

Total: 585, avg: 22.29, std: 42.44, median: 11.0, min: 4, max: 492, #>=25: 128 (21.88), #>=100: 16 (2.74), #>=200: 6 (1.03), #>=500: 0 (0.0), #>=1000: 0 (0.0)
Count of entries: 120012
--- data/new-york-latest-free/gis_osm_pois_a_free_1.shp ---
Count of entries (no points): 120012


 94%|█████████▍| 17/18 [15:19<00:13, 13.91s/it]

Total: 120011, avg: 12.64, std: 23.13, median: 9.0, min: 4, max: 4095, #>=25: 7525 (6.27), #>=100: 540 (0.45), #>=200: 131 (0.11), #>=500: 25 (0.02), #>=1000: 5 (0.0)
Count of entries: 1414963
--- data/new-york-latest-free/gis_osm_roads_free_1.shp ---
Count of entries (no points): 1414963


100%|██████████| 18/18 [20:10<00:00, 67.26s/it]

Total: 1414962, avg: 8.82, std: 15.32, median: 5.0, min: 2, max: 1623, #>=25: 84106 (5.94), #>=100: 6420 (0.45), #>=200: 1043 (0.07), #>=500: 54 (0.0), #>=1000: 7 (0.0)
